### optimized for python 3.8 - Pytorch and Tensorflow kernel on Azure on mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04 base docker image. Ensure right kernel

In [2]:
import tensorflow as tf

2023-05-08 22:13:35.342503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 22:13:41.103911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
%conda install -c conda-forge -y ipywidgets

Retrieving notices: ...working... done
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /anaconda/envs/azureml_py38_PT_TF

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.5.7   |       hbcca054_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         294 KB

The following packages will be UPDATED:

  ca-certificates                      2022.12.7-ha878542_0 --> 2023.5.7-hbcca054_0 
  certifi                            2022.12.7-pyhd8ed1ab_0 --> 2023.5.7-pyhd8ed1ab_0 



ca-certificates-2023 | 145 KB    |          

In [4]:
%pip install rouge-score

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
# %pip install tensorflow_addons

In [7]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [8]:
# %pip install tensorflow_hub
# %pip install tf_keras_vis

In [9]:
import random
import pandas as pd
from IPython.display import display, HTML


### Environment configuration finished

In [9]:
# !nvidia-smi

In [10]:
from datasets import load_dataset, load_metric
import datasets

### Load dataset

In [11]:
train_dataset = load_dataset("scientific_papers", "pubmed", split="train")

Found cached dataset scientific_papers (/home/azureuser/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


In [12]:
val_dataset = load_dataset("scientific_papers", "pubmed", split="validation")
# val_dataset_full = load_dataset("scientific_papers", "pubmed", ignore_verifications=True, split="validation[:10%]")

Found cached dataset scientific_papers (/home/azureuser/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


### show random downloaded text

In [13]:
display(train_dataset)

Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 119924
})

### Utility functions we wrote are defined in oscar_utils.py for version control. Please DO NOT edit them on the fly here. any edits and addition to a Class or method defined must be added to oscar_utils.py

In [14]:
# from oscar_utils import show_random_elements

In [15]:
def show_random_elements(dataset, num_examples=4):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    return df


In [16]:
random_df = show_random_elements(train_dataset, 1)

In [17]:
display(HTML(random_df.to_html()))

### import autotokenizer, from long-text transformer on huggingface

In [18]:
# for EDA we use  "smaller" LED checkpoint "allenai/led-base-16384". Better performance can however be attained by finetuning "allenai/led-large-16384" at the cost of a higher required GPU RAM.

In [13]:
from transformers import AutoTokenizer

In [14]:
#we start with longformer: LongformerTokenizer (AllenAI Longformer model)

tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

Pubmed's input data has a median token length of 2715 with the 90%-ile token length being 6101. The output data has a media token length of 171 with the 90%-ile token length being 352.
(cited from Big Bird: Transformers for Longer Sequences paper : https://arxiv.org/pdf/2007.14062.pdf)

Thus, we set the maximum input length to 8192 and the maximum output length to 512 to ensure that the model can attend to almost all input tokens is able to generate up to a large enough number of output tokens.

In this notebook, we are only able to train on batch_size=2 to prevent out-of-memory errors.

In [21]:
max_input_length = 8192
max_output_length = 512
batch_size = 2

Now, let's write down the input data processing function that will be used to map each data sample to the correct model format. As explained earlier article represents here our input data and abstract is the target data. The datasamples are thus tokenized up to the respective maximum lengths of 8192 and 512.

In addition to the usual attention_mask, LED can make use of an additional global_attention_mask defining which input tokens are attended globally and which are attended only locally, just as it's the case of Longformer. For more information on Longformer's self-attention, please take a look at the corresponding docs. For summarization, we follow recommendations of the paper and use global attention only for the very first token. Finally, we make sure that no loss is computed on padded tokens by setting their index to -100.

In [22]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    #Returns a dictionary containing the encoded sequence or sequence pair 
    # and additional information: the mask for sequence classification and 
    # the overflowing elements if a max_length is specified.
    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    #Hugingface tokenizer returns the attention mask and input ids.
    #     input_ids: list of token ids to be fed to a model
    # token_type_ids: list of token type ids to be fed to a model
    # attention_mask: list of indices specifying which tokens should be attended to by the model
    # overflowing_tokens: list of overflowing tokens sequences if a max length is specified and return_overflowing_tokens=True.
    # special_tokens_mask: if adding special tokens, this is a list of [0, 1], with 0 specifying special added tokens and 1 specifying sequence tokens.
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    # len(batch["input_ids"]) = minibatch size ( or batch_size)
    # len(batch["input_ids"][0]) = max_input_length
    # in effect a 2-d Python List of dimension (batch_size x max_input_length)    
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored. current pad_token_id is = 1 in this model
    # batch labels is the output tokenizer dimension (batch_size x max_output_length)
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

Downsample for EDA and development!

In [23]:
DOWNSAMPLE_COUNT = 150
train_dataset = train_dataset.select(range(DOWNSAMPLE_COUNT))
val_dataset = val_dataset.select(range(DOWNSAMPLE_COUNT))

In [24]:
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [25]:
val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Dataset to Torch format

In [26]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Load the model via the AutoModelForSeq2SeqLM class.

In [15]:
from transformers import AutoModelForSeq2SeqLM

#### Tradeoff just in EDA: We've decided to stick to the smaller model "allenai/led-base-16384" for the sake of this notebook. In addition, we directly enable gradient checkpointing and disable the caching mechanism to save memory.

In [28]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384",
                                            gradient_checkpointing=True,
                                            use_cache=False)

During training: we need to evaluate the model on Rouge

ROUGE score: https://torchmetrics.readthedocs.io/en/stable/text/rouge_score.html
Per Wikipedia: Recall-Oriented Understudy for Gisting Evaluation. It's a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.

ROUGE is currently the most well used summarization metrics (as proposed in Milestone #1) that we will use.

Setting fitting generation parameters for loss calculation.

- beam search with a small beam of just 2 to save memory. 
- force the model to generate at least 100 tokens, but no more than 512.
- setting earlyt stopping and no_repeat_ngram to prevent spurious repetition

In [29]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [30]:
rouge = load_metric("rouge")

/tmp/ipykernel_6041/4132584981.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The compute metrics function expects the generated output = pred.predictions
and true label = pred.label_ids.

We need to decode the tokens then calculate ROUGE

In [31]:
def compute_metrics(pred):
    # true labels
    labels_ids = pred.label_ids
    # predicted labels
    pred_ids = pred.predictions

    # decode in batch from tokens into string both true string and predicted string
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # compute rouge score
    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [32]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

Seq2SeqTrainer : we eventually want to turn on predict_with_generate=True to inspect. Right now I'm turning it off to save RAM.

Current batch size is set to 2, then I do gradient_accumulation_steps=4 to batch the gradient, to conserve RAM. Effective batch size becomes (batch_size = 2) * (grad_accum_steps = 4) = 8

In [33]:
#MKIMMINS: IF CUDA
# enabled floating point precision : fp16 apex training

#IF NO CUDA ON MPI, DO NOT USE FP16
CUDA = False

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=CUDA,
    output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
)

In [34]:
#MKIMMINS VAL DATASET TURNED OFF FOR SPEED
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset
    ,
    eval_dataset=val_dataset,
)

In [55]:
trainer.train()

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss


In [ ]:
# led.save("mkimmins_fine_tune_10")
import time
yyyymmddhhmm = time.strftime("%Y%m%d%H%M")

In [ ]:
print("saving model to")
print(f"./led_roberta_save{yyyymmddhhmm}")
trainer.save_model(f"./led_roberta_save{yyyymmddhhmm}")

In [27]:
from transformers import AutoModelForSeq2SeqLM

PATH = './led_roberta_save202305062343'

loaded_model = AutoModelForSeq2SeqLM.from_pretrained(PATH, local_files_only=True)

In [17]:
import torch as torch

In [28]:
loaded_model

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
      (layers): ModuleList(
        (0): LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
          )
          (

In [22]:
# load pubmed
pubmed_test = load_dataset("scientific_papers", "pubmed", ignore_verifications=True, split="test")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def generate_answer(batch):
  
  inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=8192, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to(device)
  attention_mask = inputs_dict.attention_mask.to(device)
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = loaded_model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
  batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  return batch



/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/datasets/load.py:1748: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Found cached dataset scientific_papers (/home/azureuser/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


In [23]:
device

'cpu'

In [24]:
len(pubmed_test)
DOWNSAMPLE_COUNT = 20
pubmed_test_downsample = pubmed_test.select(range(DOWNSAMPLE_COUNT))


In [30]:

result = pubmed_test_downsample.map(generate_answer, batched=True, batch_size=4)

# load rouge
rouge = load_metric("rouge")

print("Result:", rouge.compute(predictions=result["predicted_abstract"], references=result["abstract"], rouge_types=["rouge2"])["rouge2"].mid)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]